# Modeling and solving search problems

Consider the following river crossing puzzle:

## Jealous Husbands Problem with an Island

There are n couples: $(𝐻_1,𝑊_1)$, $(𝐻_2,𝑊_2)$, …, $(𝐻_𝑛,𝑊_𝑛)$  where $𝐻_𝑖$ is husband $𝑖$ and $𝑊_𝑖$ is wife $𝑖$.

They need to cross from the left bank of a river to the right bank.

There is a boat that can carry at most two people at a time.

Important: There is an island midway between the left and right banks. The boat can stop at the island.

Rules:

1. Jealousy constraint: No wife may ever be in the presence of another husband (on any shore or the island or in the boat) unless her own husband is also present. Otherwise, fights (and failures) occur.

2. The boat needs at least one person to operate (no remote-controlled trips).

3. The boat can travel between:

            Left bank ↔ Island

            Island ↔ Right bank

4. All people, including the boat, start at the left bank.

Goal: get everyone (husbands and wives) safely to the right bank, obeying all the rules.


## Tasks:

1. Put the problem in a form of a search tree: root, nodes, solution and rules for constructing the childrens.
2. Solve it usinh a classic DFS
3. Solve it using a classic Back-tracking.
4. Solve it using the A* method.  

In [9]:
from typing import Set, Tuple, List
from copy import deepcopy
from itertools import combinations

n = 3  # number of couples
husbands = {f"husband{i}" for i in range(1, n + 1)}
wives = {f"wife{i}" for i in range(1, n + 1)}
people = husbands | wives
boat_locations = ["left", "island", "right"]


class State:
    def __init__(self, left: Set[str], island: Set[str], right: Set[str], boat_location: str):
        self.left = left
        self.island = island
        self.right = right
        self.boat_location = boat_location 

    def __repr__(self):
        return f"state(L={self.left}, i={self.island}, r={self.right}, boat={self.boat_location})"

    def is_goal(self) -> bool:
        return self.right == people

    def copy(self):
        return State(set(self.left), set(self.island), set(self.right), self.boat_location)

    def __eq__(self, other):
        return (
            isinstance(other, State) and
            self.left == other.left and
            self.island == other.island and
            self.right == other.right and
            self.boat_location == other.boat_location
        )

    def __hash__(self):
        return hash((
            frozenset(self.left),
            frozenset(self.island),
            frozenset(self.right),
            self.boat_location
        ))


def is_valid_group(group: Set[str]) -> bool:
    for i in range(1, n + 1):
        wife = f"wife{i}"
        husband = f"husband{i}"
        if wife in group:
            other_husbands = {p for p in group if p.startswith("husband") and p != husband}
            if other_husbands and husband not in group:
                return False
    return True


def is_valid_state(state: State, boat_passengers: Set[str]) -> bool:
    return (
        is_valid_group(state.left) and
        is_valid_group(state.island) and
        is_valid_group(state.right) and
        is_valid_group(boat_passengers)
    )


def get_adjacent_locations(current: str) -> List[str]:
    if current == "left":
        return ["island"]
    elif current == "island":
        return ["left", "right"]
    elif current == "right":
        return ["island"]
    return []


def generate_children(state: State) -> List[Tuple[State, Set[str]]]:
    children = []
    current_location = state.boat_location
    possible_passengers = list(combinations(getattr(state, current_location), 1)) + \
                          list(combinations(getattr(state, current_location), 2))

    for passengers in possible_passengers:
        passengers = set(passengers)
        for destination in get_adjacent_locations(current_location):
            new_state = state.copy()
            getattr(new_state, current_location).difference_update(passengers)
            getattr(new_state, destination).update(passengers)
            new_state.boat_location = destination
            if is_valid_state(new_state, passengers):
                children.append((new_state, passengers))

    return children


initial_state = State(
    left=set(people),
    island=set(),
    right=set(),
    boat_location="left"
)


def state_signature(state: State) -> Tuple:
    return (
        frozenset(state.left),
        frozenset(state.island),
        frozenset(state.right),
        state.boat_location
    )


def dfs(initial_state: State):
    stack = [(initial_state, [])]
    visited = set()

    while stack:
        current_state, path = stack.pop()
        signature = state_signature(current_state)

        if signature in visited:
            continue
        visited.add(signature)

        if current_state.is_goal():
            print(f"initially:\n{initial_state}\n")
            for step_num, (state, move) in enumerate(path, 1):
                print(f"step {step_num}: move {move} → {state}")
            print(f"\nfinally:\n{current_state}")
            return

        for child_state, passengers in generate_children(current_state):
            stack.append((child_state, path + [(child_state, passengers)]))

    print("no solution found.")


dfs(initial_state)

initially:
state(L={'husband2', 'husband3', 'wife1', 'wife3', 'wife2', 'husband1'}, i=set(), r=set(), boat=left)

step 1: move {'wife2', 'wife3'} → state(L={'husband2', 'husband3', 'wife1', 'husband1'}, i={'wife2', 'wife3'}, r=set(), boat=island)
step 2: move {'wife2', 'wife3'} → state(L={'husband2', 'husband3', 'husband1', 'wife1'}, i=set(), r={'wife2', 'wife3'}, boat=right)
step 3: move {'wife3'} → state(L={'husband2', 'husband3', 'husband1', 'wife1'}, i={'wife3'}, r={'wife2'}, boat=island)
step 4: move {'wife3'} → state(L={'husband2', 'husband3', 'wife1', 'wife3', 'husband1'}, i=set(), r={'wife2'}, boat=left)
step 5: move {'husband1', 'wife1'} → state(L={'husband2', 'husband3', 'wife3'}, i={'husband1', 'wife1'}, r={'wife2'}, boat=island)
step 6: move {'husband1'} → state(L={'husband2', 'husband3', 'husband1', 'wife3'}, i={'wife1'}, r={'wife2'}, boat=left)
step 7: move {'husband2', 'husband1'} → state(L={'husband3', 'wife3'}, i={'husband2', 'husband1', 'wife1'}, r={'wife2'}, boat=isl

In [16]:
#dfs
def state_signature(state: State) -> Tuple:
       return (
           frozenset(state.left),
           frozenset(state.island),
           frozenset(state.right),
           state.boat_location
    )

def dfs(initial_state: State):
        stack = [(initial_state, [])] 
        visited = set()

        while stack:
            current_state, path = stack.pop()
            signature = state_signature(current_state)

            if signature in visited:
                continue
            visited.add(signature)

            if current_state.is_goal():
                print(f"initially:\n{initial_state}\n")
                for step_num, (state, move) in enumerate(path, 1):
                    print(f"step {step_num}: move {move} → {state}")
                print(f"\nfinally:\n{current_state}")
                return 

            for child_state, passengers in generate_children(current_state):
                stack.append((child_state, path + [(child_state, passengers)]))

            print("no solution found.")

dfs(initial_state)

no solution found.
no solution found.
no solution found.
no solution found.
no solution found.
no solution found.
no solution found.
no solution found.
no solution found.
no solution found.
no solution found.
no solution found.
no solution found.
no solution found.
no solution found.
no solution found.
no solution found.
no solution found.
no solution found.
no solution found.
no solution found.
no solution found.
no solution found.
no solution found.
no solution found.
no solution found.
no solution found.
no solution found.
no solution found.
no solution found.
no solution found.
no solution found.
initially:
state(L={'husband2', 'husband3', 'wife1', 'wife3', 'wife2', 'husband1'}, i=set(), r=set(), boat=left)

step 1: move {'wife2', 'wife3'} → state(L={'husband2', 'husband3', 'wife1', 'husband1'}, i={'wife2', 'wife3'}, r=set(), boat=island)
step 2: move {'wife2', 'wife3'} → state(L={'husband2', 'husband3', 'husband1', 'wife1'}, i=set(), r={'wife2', 'wife3'}, boat=right)
step 3: move 

In [13]:
#backtracking
from itertools import combinations

def generate_possible_moves(people):
    return [set(c) for i in [1, 2] for c in combinations(people, i)]

def backtrack(state, path, visited):
    if state.is_goal():
        return path

    visited.add(state)

    for child_state, passengers in generate_children(state):
        if child_state not in visited:
            result = backtrack(child_state, path + [(child_state, passengers)], visited)
            if result:
                return result

    return None

solution = backtrack(initial_state, [(initial_state, None)], set())

if solution:
    print(f"initially:\n{initial_state}\n")
    for step_num, (state, move) in enumerate(solution[1:], 1):  
        print(f"step {step_num}: move {sorted(move)} → {state}")
    print(f"\nfinally:\n{solution[-1][0]}")
else:
    print("no solution found.")


initially:
state(L={'husband2', 'husband3', 'wife1', 'wife3', 'wife2', 'husband1'}, i=set(), r=set(), boat=left)

step 1: move ['husband2', 'wife2'] → state(L={'husband3', 'wife1', 'wife3', 'husband1'}, i={'husband2', 'wife2'}, r=set(), boat=island)
step 2: move ['husband2'] → state(L={'husband2', 'husband3', 'wife1', 'wife3', 'husband1'}, i={'wife2'}, r=set(), boat=left)
step 3: move ['wife1'] → state(L={'husband2', 'husband3', 'wife3', 'husband1'}, i={'wife2', 'wife1'}, r=set(), boat=island)
step 4: move ['wife2'] → state(L={'husband2', 'husband3', 'wife3', 'wife2', 'husband1'}, i={'wife1'}, r=set(), boat=left)
step 5: move ['wife3'] → state(L={'husband2', 'husband3', 'wife2', 'husband1'}, i={'wife3', 'wife1'}, r=set(), boat=island)
step 6: move ['wife1'] → state(L={'husband2', 'husband3', 'wife1', 'wife2', 'husband1'}, i={'wife3'}, r=set(), boat=left)
step 7: move ['husband3'] → state(L={'husband2', 'wife1', 'wife2', 'husband1'}, i={'husband3', 'wife3'}, r=set(), boat=island)
step 8

In [18]:
#a*
import heapq
from typing import Optional, List, Tuple, Set, Dict

class Node:
    def __init__(self, state: State, parent: Optional['Node'], cost: int, heuristic: int, move: Optional[Set[str]] = None):
        self.state = state
        self.parent = parent
        self.cost = cost
        self.heuristic = heuristic
        self.move = move  

    def __lt__(self, other: 'Node'):  # Fixed: 'node' → 'Node'
        return (self.cost + self.heuristic) < (other.cost + other.heuristic)

def heuristic(state: State) -> int:
    return len(people - state.right)  # How many people are not on the right bank yet

def reconstruct_path(node: Node) -> List[Tuple[State, Optional[Set[str]]]]:
    path = []
    while node:
        path.append((node.state, node.move))
        node = node.parent
    return list(reversed(path))

def a_star_search(start_state: State) -> Optional[List[Tuple[State, Optional[Set[str]]]]]:
    open_set = []
    heapq.heappush(open_set, Node(start_state, None, 0, heuristic(start_state)))
    visited: Dict[State, int] = {start_state: 0}

    while open_set:
        current_node = heapq.heappop(open_set)
        current_state = current_node.state

        if current_state.is_goal():
            return reconstruct_path(current_node)

        for child_state, move in generate_children(current_state):
            new_cost = current_node.cost + 1
            if child_state not in visited or new_cost < visited[child_state]:
                visited[child_state] = new_cost
                heapq.heappush(open_set, Node(child_state, current_node, new_cost, heuristic(child_state), move))

    return None

solution = a_star_search(initial_state)

if solution:
    print(f"initially:\n{solution[0][0]}\n")
    for step_num, (state, move) in enumerate(solution[1:], 1):
        move_str = ', '.join(sorted(move)) if move else "None"
        print(f"step {step_num}: move {{{move_str}}} → {state}")
    print(f"\nfinally:\n{solution[-1][0]}")
else:
    print("no solution found.")


initially:
state(L={'husband2', 'husband3', 'wife1', 'wife3', 'wife2', 'husband1'}, i=set(), r=set(), boat=left)

step 1: move {wife2, wife3} → state(L={'husband2', 'husband3', 'wife1', 'husband1'}, i={'wife2', 'wife3'}, r=set(), boat=island)
step 2: move {wife2} → state(L={'husband2', 'husband3', 'wife1', 'wife2', 'husband1'}, i={'wife3'}, r=set(), boat=left)
step 3: move {wife1, wife2} → state(L={'husband2', 'husband3', 'husband1'}, i={'wife1', 'wife2', 'wife3'}, r=set(), boat=island)
step 4: move {wife1} → state(L={'husband2', 'husband3', 'wife1', 'husband1'}, i={'wife2', 'wife3'}, r=set(), boat=left)
step 5: move {husband2, husband3} → state(L={'wife1', 'husband1'}, i={'husband2', 'husband3', 'wife2', 'wife3'}, r=set(), boat=island)
step 6: move {husband2, wife2} → state(L={'husband1', 'wife1'}, i={'husband3', 'wife3'}, r={'husband2', 'wife2'}, boat=right)
step 7: move {husband2} → state(L={'husband1', 'wife1'}, i={'husband2', 'husband3', 'wife3'}, r={'wife2'}, boat=island)
step 8: